# RunnableLambda 객체 사용방법

In [13]:
from langchain_core.runnables import RunnableLambda

def func(x):
    return x * 2

runnable_1 = RunnableLambda(func)

print(runnable_1.invoke(10))

20


In [14]:
runnable_2 = RunnableLambda(lambda x: x * 2)
print(runnable_2.invoke(10))

20


In [15]:
runnable_3 = RunnableLambda(func)
print(runnable_3.batch([10, 20, 30]))

[20, 40, 60]


In [16]:
def gen(x):
    for y in x:
        yield y * 2

runnable_4 = RunnableLambda(gen)

for chunk in runnable_4.stream(range(10)):
    print(chunk)

0
2
4
6
8
10
12
14
16
18


## 순차적으로 실행하게 하기

In [17]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

chain = r1 | r2

chain.invoke(10)

32

In [18]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

chain = RunnableParallel(first = r1, second = r2)

chain.invoke(10)

{'first': 30, 'second': 12}

In [25]:
runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


## LCEL 문법 적용

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [21]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = prompt | llm
input = {
            "context": "Vector Search에 의한 컨텐스트 내용", 
            "query": "안녕"
        }
print(chain.invoke(input))

content='안녕하세요! 무엇을 도와드릴까요? Vector Search에 대해 궁금한 점이 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 42, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BkKpVB6aqhj2ROs2665sHwxrOTdfJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--8f492689-450c-421a-a23c-6f28cf4fe470-0' usage_metadata={'input_tokens': 42, 'output_tokens': 25, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

# 기본값을 제공하는 함수
def add_default_values(input_dict):
    return {
        "context": input_dict.get("context", "Vector Search에 의한 컨텐스트 내용"),
        "query": input_dict.get("query", "안녕")
    }


# 체인 구성
chain = RunnableLambda(add_default_values) | prompt | llm

In [23]:
result = chain.invoke({})
print(result)

content='안녕하세요! 어떻게 도와드릴까요? Vector Search에 관련된 질문이 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 42, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BkKpWxb8RSxfpNa1niryaAa05l3Bj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a91e9beb-62d4-4480-9108-8d58a6d89426-0' usage_metadata={'input_tokens': 42, 'output_tokens': 22, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [24]:
result2 = chain.invoke({"query": "RunnableLambda에 대해 1문장으로 설명해줘"})
print(result2)

content='RunnableLambda는 자바에서 람다 표현식을 사용하여 Runnable 인터페이스의 인스턴스를 간단하고 명확하게 생성할 수 있게 해주는 기능입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 52, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BkKpYuYN92gJ1GRjViIMx0KRxooNY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9a4c6f7f-fb29-4f31-a0bc-e8608c7f81ea-0' usage_metadata={'input_tokens': 52, 'output_tokens': 37, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
